In [1]:
!pip install segmentation_models

     |████████████████████████████████| 50 kB 8.2 MB/s 


In [2]:
import os
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

path = "/content/drive/My Drive/Master_Project"
path_prefix = path

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['TransUNet-single',
 'data',
 'predictions',
 'Xception_Gray_flow.ipynb',
 'Xception_Event_flow.ipynb',
 'Xception_Dual_flow.ipynb',
 'old_loss',
 'Grayscale_output',
 'Event_output',
 'Xcep_loss_record_train_Gray.npy',
 'Xcep_loss_record_train_Event.npy',
 'Xcep_iou_record_test_Gray.npy',
 'Xception_Grayscale.ipynb',
 'Xcep_iou_record_test_Event.npy',
 'Xception_Event.ipynb',
 'Dual_output',
 'Xcep_loss_record_train_Dual.npy',
 'Xcep_iou_record_test_Dual.npy',
 'Xception_Final_old.ipynb',
 'Xception_Final.ipynb',
 'Xception_Dual_try.ipynb',
 'Xception_Dual.ipynb',
 'Xcep_iou_record_test_Final.npy',
 'loss_plot',
 'predict_helper.ipynb',
 'Final_output',
 'Xcep_loss_record_train_Final.npy',
 'Xception_Final_flowdir.ipynb']

In [3]:
# Set GPU environment
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# ==============================================================================
# =                                   param                                    =
# ==============================================================================
import numpy as np
import tensorflow as tf
import tqdm
import cv2
import copy
import random
from tensorflow.keras import layers
from tensorflow.keras import Model
from segmentation_models.losses import bce_jaccard_loss

# ==============================================================================
# =                                   model                                    =
# ==============================================================================
def xception_final(Input_Shape, NUM_CHANNELS):
    input = tf.keras.layers.Input(Input_Shape)
    # s = tf.keras.layers.Lambda(lambda x: x / 255)(input)

    base_model = tf.keras.applications.xception.Xception(
        include_top=False, weights=None, input_tensor=input,
        input_shape=(320, 320, NUM_CHANNELS), pooling=max
    )
    output_1 = base_model.get_layer('block2_sepconv2_bn').output
    output_2 = base_model.get_layer('block3_sepconv2_bn').output
    output_3 = base_model.get_layer('block4_sepconv2_bn').output
    output_4 = base_model.get_layer('block13_sepconv2_bn').output
    output_5 = base_model.get_layer('block14_sepconv2_bn').output

    decoder_0 = tf.keras.layers.Conv2DTranspose(filters=2048, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(output_5)
    decoder_0 = layers.BatchNormalization()(decoder_0)
    decoder_0 = tf.image.resize(decoder_0, (tf.shape(output_5)[1], tf.shape(output_5)[2]))
    decoder_0 = tf.concat([decoder_0, output_5], 3)

    decoder_1 = tf.keras.layers.Conv2D(filters=2048, kernel_size=3, activation='relu', padding='same')(decoder_0)
    decoder_1 = tf.keras.layers.Conv2DTranspose(filters=1024, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_1)
    decoder_1 = layers.BatchNormalization()(decoder_1)
    decoder_1 = tf.image.resize(decoder_1, (tf.shape(output_4)[1], tf.shape(output_4)[2]))
    decoder_1 = tf.concat([decoder_1, output_4], 3)
    #
    decoder_2 = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, activation='relu', padding='same')(decoder_1)
    decoder_2 = tf.keras.layers.Conv2DTranspose(filters=728, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_2)
    decoder_2 = layers.BatchNormalization()(decoder_2)
    decoder_2 = tf.image.resize(decoder_2, (tf.shape(output_3)[1], tf.shape(output_3)[2]))
    decoder_2 = tf.concat([decoder_2, output_3], 3)
    #
    decoder_3 = tf.keras.layers.Conv2D(filters=728, kernel_size=3, activation='relu', padding='same')(decoder_2)
    decoder_3 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_3)
    decoder_3 = layers.BatchNormalization()(decoder_3)
    decoder_3 = tf.image.resize(decoder_3, (tf.shape(output_2)[1], tf.shape(output_2)[2]))
    decoder_3 = tf.concat([decoder_3, output_2], 3)
    #
    decoder_4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(decoder_3)
    decoder_4 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_4)
    decoder_4 = layers.BatchNormalization()(decoder_4)
    decoder_4 = tf.image.resize(decoder_4, (tf.shape(output_1)[1], tf.shape(output_1)[2]))
    decoder_4 = tf.concat([decoder_4, output_1], 3)
    decoder_4 = tf.image.resize(decoder_4, [320, 320])
    #
    output = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(decoder_4)
    #
    model = Model(inputs=base_model.input, outputs=output)

    # model.compile(optimizer='adam', loss=bce_jaccard_loss, metrics=[iou_score])

    return model


Segmentation Models: using `keras` framework.


In [6]:
#=============== Training Process =====================

def my_mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)


def train(dataset_dir, load_size, batch_size, epochs, epoch_decay, lr, beta_1, NUM_CHANNELS):
    # output_dir
    output_dir = os.path.join('Final_output')
    my_mkdir(output_dir)

    models_path_final = os.path.join('Final_output', 'models')
    my_mkdir(models_path_final)


    # ==============================================================================
    # =                                    metrics                                 =
    # ==============================================================================

    def calc_IoU(mask_1, mask_2):
        mask_1 = mask_1 > 0.3
        mask_2 = mask_2 > 0.3

        TP = mask_1 * mask_2
        TP = TP.sum()

        FP = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_2
        FP = FP.sum()

        FN = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_1
        FN = FN.sum()

        return TP / (TP + FP + FN)

    # ==============================================================================
    # =                                    data                                    =
    # ==============================================================================

    # loss record path
    loss_path = '/content/drive/My Drive/Master_Project/TransUNet-single/loss_record'
    loss_gray = np.load(loss_path+'/loss_record_test_GaE_gray.npy', allow_pickle=True)
    loss_event = np.load(loss_path+'/loss_record_test_GaE_event.npy', allow_pickle=True)
    

    pred_dir = "/content/drive/My Drive/Master_Project/predictions/"
    N_CH = NUM_CHANNELS // 2


    def generator_train(BS):

        gray_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)
        gray_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)
        
        event_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)
        event_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)
        train_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                       dtype='float32', rescale=1. / 255,
                                                                       brightness_range=None, fill_mode='constant',
                                                                       cval=125, zoom_range=[0.9, 1.1],
                                                                       rotation_range=180)

        gray_img_iterator = gray_image_gen.flow_from_directory(
            dataset_dir + "Beta_training/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        gray_pred_iterator = gray_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_Gray320/Beta_training",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_img_iterator = event_image_gen.flow_from_directory(
            dataset_dir + "Beta_training/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_pred_iterator = event_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_EvB320/Beta_training",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')

        train_gt_generator = train_gt_gen.flow_from_directory(
            dataset_dir + "Beta_training/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='nearest')

        while True:
            gray_img = gray_img_iterator.next()
            gray_pred = gray_pred_iterator.next()
            event_img = event_img_iterator.next()
            event_pred = event_pred_iterator.next()
            gt = train_gt_generator.next()

            # channels concat
            idx = (gray_img_iterator.batch_index - 1) * gray_img_iterator.batch_size
            gray_slice_names = [gray_img_iterator.filenames[gray_img_iterator.index_array[i]] for i in range(idx, idx + gray_img_iterator.batch_size)]
            
            idx = (event_img_iterator.batch_index - 1) * event_img_iterator.batch_size
            event_slice_names = [event_img_iterator.filenames[event_img_iterator.index_array[i]] for i in range(idx, idx + event_img_iterator.batch_size)]
            
            images = []

            for num in range(len(gray_img)):
                idx_gray = np.where(loss_gray[0]==gray_slice_names[num][11:-4])
                idx_event = np.where(loss_event[0]==event_slice_names[num][11:-4])
                Loss_1 = float(loss_gray[2][idx_gray])
                Loss_2 = float(loss_event[2][idx_event])
                if (Loss_1+Loss_2) == 0 :
                    Loss_1 = 0.5
                    Loss_2 = 0.5
                N_Gray = round(N_CH * Loss_1 / (Loss_1+Loss_2))
                N_Event = N_CH - N_Gray

                image_cc = np.zeros(shape=gray_img[num,:,:,:].shape)    # initial zero slice for concat convenience

                for _ in range(N_Gray):
                    image_cc = np.concatenate((image_cc, gray_img[num,:,:,:], gray_pred[num,:,:,:]), axis=-1)    # concat grayscale
                for _ in range(N_Event):
                    image_cc = np.concatenate((image_cc, event_img[num,:,:,:], event_pred[num,:,:,:]), axis=-1)    # concat event

                image_cc = image_cc[:,:,1:]    # remove zero slice
                images.append(image_cc)

            yield np.array(images), gt


    def generator_val(BS=1):

        gray_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        gray_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        event_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        event_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        val_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)

        gray_img_iterator = gray_image_gen.flow_from_directory(
            dataset_dir + "Beta_validation/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        gray_pred_iterator = gray_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_Gray320/Beta_validation",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_img_iterator = event_image_gen.flow_from_directory(
            dataset_dir + "Beta_validation/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_pred_iterator = event_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_EvB320/Beta_validation",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')

        val_gt_generator = val_gt_gen.flow_from_directory(
            dataset_dir + "Beta_validation/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='nearest')

        while True:
            gray_img = gray_img_iterator.next()
            gray_pred = gray_pred_iterator.next()
            event_img = event_img_iterator.next()
            event_pred = event_pred_iterator.next()
            gt = val_gt_generator.next()

            # channels concat
            idx = (gray_img_iterator.batch_index - 1) * gray_img_iterator.batch_size
            gray_slice_names = [gray_img_iterator.filenames[gray_img_iterator.index_array[i]] for i in range(idx, idx + gray_img_iterator.batch_size)]
            
            idx = (event_img_iterator.batch_index - 1) * event_img_iterator.batch_size
            event_slice_names = [event_img_iterator.filenames[event_img_iterator.index_array[i]] for i in range(idx, idx + event_img_iterator.batch_size)]
            
            images = []

            for num in range(BS):
                idx_gray = np.where(loss_gray[0]==gray_slice_names[num][11:-4])
                idx_event = np.where(loss_event[0]==event_slice_names[num][11:-4])
                Loss_1 = float(loss_gray[2][idx_gray])
                Loss_2 = float(loss_event[2][idx_event])
                if (Loss_1+Loss_2) == 0 :
                    Loss_1 = 0.5
                    Loss_2 = 0.5
                N_Gray = round(N_CH * Loss_1 / (Loss_1+Loss_2))
                N_Event = N_CH - N_Gray

                image_cc = np.zeros(shape=gray_img[num,:,:,:].shape)    # initial zero slice for concat convenience

                for _ in range(N_Gray):
                    image_cc = np.concatenate((image_cc, gray_img[num,:,:,:], gray_pred[num,:,:,:]), axis=-1)    # concat grayscale
                for _ in range(N_Event):
                    image_cc = np.concatenate((image_cc, event_img[num,:,:,:], event_pred[num,:,:,:]), axis=-1)    # concat event

                image_cc = image_cc[:,:,1:]    # remove zero slice
                images.append(image_cc)

            yield np.array(images), gt

    # ==============================================================================
    # =                                   models                                   =
    # ==============================================================================
    len_train_set = 420
    len_val_set = 60

    final_segmentation_model = xception_final((320, 320, NUM_CHANNELS), NUM_CHANNELS)


    model_optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1)

    # ==============================================================================
    # =                                 train step                                 =
    # ==============================================================================

    @tf.function
    def train_step(img, gt):
        img_ = tf.convert_to_tensor(img)
        gt_ = tf.convert_to_tensor(gt)
        with tf.GradientTape(persistent=True) as t_:
            final_prediction = final_segmentation_model(img_, training=True)
            final_loss = bce_jaccard_loss(gt_, final_prediction)

        model_grad = t_.gradient(final_loss, final_segmentation_model.trainable_variables)
        model_optimizer.apply_gradients(zip(model_grad, final_segmentation_model.trainable_variables))

        return final_loss, final_prediction

    @tf.function
    def sample(img):
        img_ = tf.convert_to_tensor(img)
        final_prediction = final_segmentation_model(img_)

        return final_prediction



    # ==============================================================================
    # =                                    run                                     =
    # ==============================================================================

    t_generator = generator_train(batch_size)
    v_generator = generator_val()

    best_val_metric = np.NINF

    # epoch counter
    ep_cnt = tf.Variable(initial_value=0, trainable=False, dtype=tf.int64)

    # summary
    train_summary_writer = tf.summary.create_file_writer(os.path.join(output_dir, 'summaries', 'train'))

    # sample
    sample_dir = os.path.join(output_dir, 'samples_training')
    my_mkdir(sample_dir)

    # main loop
    with train_summary_writer.as_default():
        loss_record_train = [[],[],[]]
        for ep in tqdm.trange(epochs, desc='Epoch Loop'):
            # update epoch counter
            ep_cnt.assign_add(1)
            # train for an epoch
            t = tqdm.tqdm(range(int(len_train_set / batch_size)))

            a = 0

            for _ in t:
                img, gt = next(t_generator)
                loss, pred_temp = train_step(img, gt)
                iou = calc_IoU(pred_temp.numpy(), gt)
                loss_record_train[0].append(loss)
                loss_record_train[1].append(iou)
                t.set_description("Training loss: {}, ".format(str(np.array(loss))) + "IoU: {} |".format(str(np.array(iou))))

            val_metric = []
            for _ in range(len_val_set):
                img, gt = next(v_generator)
                prediction = sample(img).numpy()
                val_metric.append(calc_IoU(prediction, gt))

            val_metric = np.mean(np.array(val_metric))
            loss_record_train[2].append(val_metric)


            img = np.concatenate(
                (cv2.cvtColor(img[0,:,:,0].astype('float32'), cv2.COLOR_GRAY2RGB), cv2.cvtColor(img[0,:,:,1].astype('float32'), cv2.COLOR_GRAY2RGB), cv2.cvtColor(prediction[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(gt[0], cv2.COLOR_GRAY2RGB)),
                axis=1) * 255

            cv2.imwrite(os.path.join(sample_dir, 'epoch_{}-iter_{}.jpg'.format(str(ep).zfill(3),
                          str(model_optimizer.iterations.numpy()).zfill(9))), img)

            if val_metric >= best_val_metric:
                print('Validation metric improved from {} to {}'.format(str(best_val_metric), str(val_metric)))
                best_val_metric = copy.deepcopy(val_metric)
                final_segmentation_model.save_weights(os.path.join(models_path_final, 'model_{}.h5'.format(str(ep).zfill(3))),
                                                          overwrite=True, save_format=None)                                                                                                                  
            else:
                print('Validation metric did not improve from {}'.format(str(best_val_metric)))

        np.save("/content/drive/My Drive/Master_Project/Xcep_loss_record_train_Final.npy", loss_record_train)


dataset_dir = "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset_final/"
load_size = [320, 320]
batch_size = 8
epochs = 150
epoch_decay = 25
lr = 0.001
beta_1 = 0.9
NUM_CHANNELS = 8
train(dataset_dir, load_size, batch_size, epochs, epoch_decay, lr, beta_1, NUM_CHANNELS)


/usr/local/lib/python3.7/dist-packages/keras/applications/xception.py:133: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 8 input channels.
  weights=weights)
  0%|          | 0/26 [00:00<?, ?it/s]

Found 420 images belonging to 1 classes.
Found 420 images belonging to 1 classes.
Found 420 images belonging to 1 classes.
Found 420 images belonging to 1 classes.
Found 420 images belonging to 1 classes.



Training loss: 1.9587209, IoU: 0.1418688358031422 |:   0%|          | 0/26 [01:18<?, ?it/s]
Training loss: 1.9587209, IoU: 0.1418688358031422 |:   4%|▍         | 1/26 [01:18<32:51, 78.85s/it]
Training loss: 1.0874667, IoU: 0.6800378472300732 |:   4%|▍         | 1/26 [01:44<32:51, 78.85s/it]
Training loss: 1.0874667, IoU: 0.6800378472300732 |:   8%|▊         | 2/26 [01:44<18:55, 47.33s/it]
Training loss: 0.862264, IoU: 0.8067213885751268 |:   8%|▊         | 2/26 [02:09<18:55, 47.33s/it] 
Training loss: 0.862264, IoU: 0.8067213885751268 |:  12%|█▏        | 3/26 [02:09<14:21, 37.44s/it]
Training loss: 0.70265603, IoU: 0.8678280286856567 |:  12%|█▏        | 3/26 [02:34<14:21, 37.44s/it]
Training loss: 0.70265603, IoU: 0.8678280286856567 |:  15%|█▌        | 4/26 [02:34<11:54, 32.46s/it]
Training loss: 0.7245443, IoU: 0.8632687427912341 |:  15%|█▌        | 4/26 [03:00<11:54, 32.46s/it] 
Training loss: 0.7245443, IoU: 0.8632687427912341 |:  19%|█▉        | 5/26 [03:00<10:30, 30.00s/it]
Train

Found 60 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Validation metric improved from -inf to 0.06311667868406572


  0%|          | 0/26 [00:00<?, ?it/s]
Training loss: 0.4672638, IoU: 0.9013289246641552 |:   0%|          | 0/26 [00:16<?, ?it/s]
Training loss: 0.4672638, IoU: 0.9013289246641552 |:   4%|▍         | 1/26 [00:16<06:54, 16.58s/it]
Training loss: 0.5939432, IoU: 0.8840634408694384 |:   4%|▍         | 1/26 [00:19<06:54, 16.58s/it]
Training loss: 0.5939432, IoU: 0.8840634408694384 |:   8%|▊         | 2/26 [00:19<03:24,  8.51s/it]
Training loss: 0.5108085, IoU: 0.9110326229625564 |:   8%|▊         | 2/26 [00:22<03:24,  8.51s/it]
Training loss: 0.5108085, IoU: 0.9110326229625564 |:  12%|█▏        | 3/26 [00:22<02:16,  5.95s/it]
Training loss: 0.49417168, IoU: 0.9285179329283515 |:  12%|█▏        | 3/26 [00:25<02:16,  5.95s/it]
Training loss: 0.49417168, IoU: 0.9285179329283515 |:  15%|█▌        | 4/26 [00:25<01:43,  4.72s/it]
Training loss: 0.5567972, IoU: 0.9011840209921738 |:  15%|█▌        | 4/26 [00:28<01:43,  4.72s/it] 
Training loss: 0.5567972, IoU: 0.9011840209921738 |:  19%|█▉      

...


  0%|          | 0/26 [00:00<?, ?it/s]
Training loss: 0.5032823, IoU: 0.9593187223712664 |:   0%|          | 0/26 [00:02<?, ?it/s]
Training loss: 0.5032823, IoU: 0.9593187223712664 |:   4%|▍         | 1/26 [00:02<01:09,  2.77s/it]
Training loss: 0.43566132, IoU: 0.9572132355793476 |:   4%|▍         | 1/26 [00:05<01:09,  2.77s/it]
Training loss: 0.43566132, IoU: 0.9572132355793476 |:   8%|▊         | 2/26 [00:05<01:06,  2.77s/it]
Training loss: 0.47698092, IoU: 0.9605779355320812 |:   8%|▊         | 2/26 [00:08<01:06,  2.77s/it]
Training loss: 0.47698092, IoU: 0.9605779355320812 |:  12%|█▏        | 3/26 [00:08<01:03,  2.76s/it]
Training loss: 0.4903623, IoU: 0.9619531448172824 |:  12%|█▏        | 3/26 [00:11<01:03,  2.76s/it] 
Training loss: 0.4903623, IoU: 0.9619531448172824 |:  15%|█▌        | 4/26 [00:11<01:01,  2.78s/it]
Training loss: 0.42698035, IoU: 0.9489999110792553 |:  15%|█▌        | 4/26 [00:13<01:01,  2.78s/it]
Training loss: 0.42698035, IoU: 0.9489999110792553 |:  19%|█▉ 

Validation metric did not improve from 0.835755820272055



  0%|          | 0/26 [00:00<?, ?it/s]
Training loss: 0.43752682, IoU: 0.956277063822688 |:   0%|          | 0/26 [00:02<?, ?it/s]
Training loss: 0.43752682, IoU: 0.956277063822688 |:   4%|▍         | 1/26 [00:02<01:09,  2.78s/it]
Training loss: 0.46679842, IoU: 0.964485783028109 |:   4%|▍         | 1/26 [00:05<01:09,  2.78s/it]
Training loss: 0.46679842, IoU: 0.964485783028109 |:   8%|▊         | 2/26 [00:05<01:06,  2.78s/it]
Training loss: 0.4700362, IoU: 0.9673780883963543 |:   8%|▊         | 2/26 [00:08<01:06,  2.78s/it]
Training loss: 0.4700362, IoU: 0.9673780883963543 |:  12%|█▏        | 3/26 [00:08<01:03,  2.77s/it]
Training loss: 0.4611624, IoU: 0.9619150109512763 |:  12%|█▏        | 3/26 [00:11<01:03,  2.77s/it]
Training loss: 0.4611624, IoU: 0.9619150109512763 |:  15%|█▌        | 4/26 [00:11<01:00,  2.77s/it]
Training loss: 0.4485189, IoU: 0.9591627405643156 |:  15%|█▌        | 4/26 [00:13<01:00,  2.77s/it]
Training loss: 0.4485189, IoU: 0.9591627405643156 |:  19%|█▉        

Validation metric did not improve from 0.835755820272055



/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [4]:
#================== Testing Process ==================
def my_mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def test(dataset_dir, load_size, NUM_CHANNELS):
    # output_dir
    output_dir = os.path.join('Final_output')
    my_mkdir(output_dir)


    # ==============================================================================
    # =                                    metrics                                 =
    # ==============================================================================

    def calc_IoU(mask_1, mask_2):
        mask_1 = mask_1 > 0.3
        mask_2 = mask_2 > 0.3

        TP = mask_1 * mask_2
        TP = TP.sum()

        FP = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_2
        FP = FP.sum()

        FN = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_1
        FN = FN.sum()

        return TP / (TP + FP + FN)

    # ==============================================================================
    # =                                    data                                    =
    # ==============================================================================

    # loss record path
    loss_path = '/content/drive/My Drive/Master_Project/TransUNet-single/loss_record'
    loss_gray = np.load(loss_path+'/loss_record_test_GaE_gray.npy', allow_pickle=True)
    loss_event = np.load(loss_path+'/loss_record_test_GaE_event.npy', allow_pickle=True)
    

    pred_dir = "/content/drive/My Drive/Master_Project/predictions/"
    N_CH = NUM_CHANNELS // 2

    def test_gen(BS=1):

        gray_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        gray_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        event_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        event_pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        test_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)

        gray_img_iterator = gray_image_gen.flow_from_directory(
            dataset_dir + "Testing/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        gray_pred_iterator = gray_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_Gray320/Testing",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_img_iterator = event_image_gen.flow_from_directory(
            dataset_dir + "Testing/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        event_pred_iterator = event_pred_gen.flow_from_directory(
            pred_dir + "TU_GaE_final_EvB320/Testing",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')

        test_gt_generator = test_gt_gen.flow_from_directory(
            dataset_dir + "Testing/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='nearest')

        while True:
            gray_img = gray_img_iterator.next()
            gray_pred = gray_pred_iterator.next()
            event_img = event_img_iterator.next()
            event_pred = event_pred_iterator.next()
            gt = test_gt_generator.next()

            # channels concat
            idx = (gray_img_iterator.batch_index - 1) * gray_img_iterator.batch_size
            gray_slice_names = [gray_img_iterator.filenames[gray_img_iterator.index_array[i]] for i in range(idx, idx + gray_img_iterator.batch_size)]
            
            idx = (event_img_iterator.batch_index - 1) * event_img_iterator.batch_size
            event_slice_names = [event_img_iterator.filenames[event_img_iterator.index_array[i]] for i in range(idx, idx + event_img_iterator.batch_size)]
            
            images = []

            for num in range(BS):
                idx_gray = np.where(loss_gray[0]==gray_slice_names[num][11:-4])
                idx_event = np.where(loss_event[0]==event_slice_names[num][11:-4])
                Loss_1 = float(loss_gray[2][idx_gray])
                Loss_2 = float(loss_event[2][idx_event])
                if (Loss_1+Loss_2) == 0 :
                    Loss_1 = 0.5
                    Loss_2 = 0.5
                N_Gray = round(N_CH * Loss_1 / (Loss_1+Loss_2))
                N_Event = N_CH - N_Gray

                image_cc = np.zeros(shape=gray_img[num,:,:,:].shape)    # initial zero slice for concat convenience

                for _ in range(N_Gray):
                    image_cc = np.concatenate((image_cc, gray_img[num,:,:,:], gray_pred[num,:,:,:]), axis=-1)    # concat grayscale
                for _ in range(N_Event):
                    image_cc = np.concatenate((image_cc, event_img[num,:,:,:], event_pred[num,:,:,:]), axis=-1)    # concat event

                image_cc = image_cc[:,:,1:]    # remove zero slice
                images.append(image_cc)

            yield np.array(images), gt


    # ==============================================================================
    # =                                   models                                   =
    # ==============================================================================
    len_test_set = 200

    final_segmentation_model = xception_final((320, 320, NUM_CHANNELS), NUM_CHANNELS)

    model_path = path + "/Final_output/models"
    all_models = os.listdir(model_path)
    all_models.sort()

    final_segmentation_model.load_weights(model_path + '/' + all_models[-1], by_name=False)



    # ==============================================================================
    # =                                 train step                                 =
    # ==============================================================================

    @tf.function
    def evaluate(img):
        img_ = tf.convert_to_tensor(img)
        final_prediction = final_segmentation_model(img_)

        return final_prediction



    # ==============================================================================
    # =                                    run                                     =
    # ==============================================================================

    te_generator = test_gen()


    # summary
    test_summary_writer = tf.summary.create_file_writer(os.path.join(output_dir, 'summaries', 'test'))

    # sample
    evaluate_dir = os.path.join(output_dir, 'test_prediction')
    my_mkdir(evaluate_dir)

    # main loop
    with test_summary_writer.as_default():
        iou_record_test = []
        for i in tqdm.trange(len_test_set, desc='Testing images'):
            img, gt = next(te_generator)
            prediction = evaluate(img).numpy()
            iou = calc_IoU(prediction, gt)
            iou_record_test.append(iou)
            print(" | Case {} ".format(str(np.array(i))) + "evaluated IoU: {}".format(str(np.array(iou))))

            img = np.concatenate(
                (cv2.cvtColor(img[0,:,:,0].astype('float32'), cv2.COLOR_GRAY2RGB), cv2.cvtColor(img[0,:,:,1].astype('float32'), cv2.COLOR_GRAY2RGB), cv2.cvtColor(prediction[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(gt[0], cv2.COLOR_GRAY2RGB)),
                axis=1) * 255

            cv2.imwrite(os.path.join(evaluate_dir, 'Image Test Case_{}.jpg'.format(str(i).zfill(3))), img)
            
        np.save("/content/drive/My Drive/Master_Project/Xcep_iou_record_test_Final.npy", iou_record_test)


dataset_dir = "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset_final/"
load_size = [320, 320]
NUM_CHANNELS = 8
test(dataset_dir, load_size, NUM_CHANNELS)

/usr/local/lib/python3.7/dist-packages/keras/applications/xception.py:133: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 8 input channels.
  weights=weights)
Testing images:   0%|          | 0/200 [00:00<?, ?it/s]

Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


Testing images:   0%|          | 1/200 [00:32<1:47:55, 32.54s/it]

 | Case 0 evaluated IoU: 0.6741022850924918


Testing images:   1%|          | 2/200 [00:36<52:46, 15.99s/it]  

...

 | Case 197 evaluated IoU: 0.8190784155214228


Testing images: 100%|█████████▉| 199/200 [12:13<00:03,  3.40s/it]

 | Case 198 evaluated IoU: 0.8685521688159438


Testing images: 100%|██████████| 200/200 [12:17<00:00,  3.69s/it]

 | Case 199 evaluated IoU: 0.8547150078438208
